<a href="https://colab.research.google.com/github/tempdata73/crowd_analysis/blob/master/train_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CSRNet: Congested Scene Recognition Network
The purpose of this notebook is to train a CSRNet with the help of GPU instances.

In [0]:
!pip install PyDrive -qq

In [0]:
import os
from getpass import getpass

user = input('User: ')
password = getpass('Password: ')
GIT_AUTH = user + ':' + password

!git clone https://$GIT_AUTH@github.com/$user/crowd_estimation.git

User: tempdata73
Password: ··········
Cloning into 'crowd_estimation'...
remote: Enumerating objects: 67, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 67 (delta 27), reused 55 (delta 18), pack-reused 0
Unpacking objects: 100% (67/67), done.


In [0]:
%cd crowd_estimation

/content/crowd_estimation


In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
FILE_ID = '1FKTNDuK8-IlvufVMW8ymP_onPYhJNYL1'
FILENAME = 'shanghai_dataset.zip'
DST_DIR = 'Shanghai'

download = drive.CreateFile({'id': FILE_ID})
download.GetContentFile(FILENAME)

In [0]:
from zipfile import ZipFile

ZipFile(FILENAME).extractall(DST_DIR)

In [0]:
!rm shanghai_dataset.zip

## Density map generation

In [0]:
import time
start_time = time.time()
!python make_dataset.py Shanghai/ Shanghai_A
end_time = time.time()
print('Density map generation took {:0.2f} secs'.format(end_time - start_time))

INFO:root:Creating Shanghai/part_B_final/train_data/densities directory
INFO:root:Processed 0 images
INFO:root:Processed 100 images
INFO:root:Processed 200 images
INFO:root:Processed 300 images
INFO:root:Creating Shanghai/part_B_final/test_data/densities directory
INFO:root:Processed 0 images
INFO:root:Processed 100 images
INFO:root:Processed 200 images
INFO:root:Processed 300 images
Density map generation took 106.35 secs


In [0]:
!python train.py part_A/trainval.json part_A/test.json

INFO:root:Using cuda:0 device for training
INFO:root:Building model
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/checkpoints/vgg16-397923af.pth
100% 528M/528M [00:24<00:00, 22.8MB/s]
INFO:root:Training model
INFO:root:Epoch [0/400] (0/400) Time = 0.02, Total time = 0.02 Current loss = 11.037, Avg loss = 11.037
INFO:root:Epoch [0/400] (100/400) Time = 0.00, Total time = 0.27 Current loss = 1.951, Avg loss = 12.701
INFO:root:Epoch [0/400] (200/400) Time = 0.00, Total time = 0.53 Current loss = 2.381, Avg loss = 14.080
INFO:root:Epoch [0/400] (300/400) Time = 0.00, Total time = 0.77 Current loss = 3.519, Avg loss = 15.161
INFO:root:Evaluating model...
INFO:root:Mean average Error (MAE) = 75.9799
INFO:root:Checkpoint created
INFO:root:Saving loss data in data/loss_data.json
INFO:root:Epoch [1/400] (0/400) Time = 0.00, Total time = 0.00 Current loss = 26.772, Avg loss = 26.772
INFO:root:Epoch [1/400] (100/400) Time = 0.00, Total time = 0.25 Cur

## Model tests

In [0]:
import torch
import os

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
CKPT_DIR = 'ckpts'
models = []

for model_fn in os.listdir(CKPT_DIR):
  if model_fn == 'model.pth.tar':
    continue
  model_path = os.path.join(CKPT_DIR, model_fn)
  ckpt = torch.load(model_path, map_location=torch.device(device))
  models.append((model_path, ckpt['loss']))

models = sorted(models, key=lambda loss: loss[1], reverse=False)
model_path = models[0][0]
print(model_path)

ckpts/model-8.17.pth.tar


In [0]:
!python validation.py $model_path part_A/test.json

INFO:root:Evaluating using cuda:0 device
INFO:root:Evaluating on 316 images
INFO:root:Building model
INFO:root:Starting evaluation
INFO:root:Image:0:Error:2.2095:SquaredError:4.8819	MAE = 2.2095	MSE = 2.2095
INFO:root:Image:100:Error:3.6945:SquaredError:13.6497	MAE = 9.5756	MSE = 14.9662
INFO:root:Image:200:Error:2.9186:SquaredError:8.5181	MAE = 10.3117	MSE = 15.8385
INFO:root:Image:300:Error:2.4286:SquaredError:5.8980	MAE = 10.5338	MSE = 16.2580
INFO:root:Metrics saved in data/metrics.json


In [0]:
import json

with open('data/metrics.json') as infile:
  data = json.load(infile)

data

{'mae': 10.611205546161797,
 'mse': 16.28243121657331,
 'ssim': 0.976537851188304}

In [0]:
# Last epoch did not necessarily yield best model
# Fix data according to that

ckpt = torch.load(model_path, map_location=torch.device(device))
last_epoch = ckpt['epoch']

with open('data/loss_data.json') as infile:
  loss = json.load(infile)

last_ckpt_loss_data = {'train_mae': loss['train_mae'][:last_epoch], 'val_mae': loss['val_mae'][:last_epoch]}

with open('data/loss_data.json', 'w') as outfile:
  json.dump(last_ckpt_loss_data, outfile)

In [0]:
# Download model and data
from google.colab import files

files.download(model_path)
files.download('data/metrics.json')
files.download('data/loss_data.json')